# AI-CA2-Genetics
## Ali Padyav - 810199388

Following constants are probability of our operation that like `sample size / populaion size`

In [1]:
crossoverProbability = 0.4
mutationProbability = 0.4
carryPercentage = 0.3
populationSize = 100
generations = 10000

## Steps
* `makeFirstPopulation`:
We randomly fill operators and operands by their position.

* `findEquation`: We do some stuffs repeatedly in expected generations to rise up average fitness.

    1. `createMatingPool`: Fitness function is $$fitness(chromosome)= 1/(|equationResult - goalNumber|+1)$$
    It means better equation would gets more fittness. Chromosomes with higher fittness have higher chance to be in mating pool.

    2. `createCrossoverPool`: We pass mating pool to this function to crossover chromosomes by excpected rate. Crossover is done by splitting chromosomes to 3 parts and swapping middle part.

    3. `createMutatedPool`: After crossover we mutate chromosomes and change it's data randomly.

* If our calculation has been done in expected generations, we print result.

In [2]:
import random
import bisect
import time
from itertools import accumulate
from copy import deepcopy

class EquationBuilder:

    def __init__(self, operators, operands, equationLength, goalNumber):
        self.operators = operators
        self.operands = operands
        self.equationLength = equationLength
        self.goalNumber = goalNumber
        self.gen = 0

        # Create the earliest population at the begining
        self.population = self.makeFirstPopulation()

    def make_chromosome(self):
        chromosome = []
        for i in range(self.equationLength):
            if i % 2:
                chromosome.append(random.choice(self.operators))
            else:
                chromosome.append(str(random.choice(self.operands)))
        return chromosome

    def makeFirstPopulation(self):
        population = []
        for _ in range(populationSize):
            population.append(self.make_chromosome())
        return population


    def findEquation(self):
        # Create a new generation of chromosomes, and make it better in every iteration
        while (True):
            self.gen += 1
            
            random.shuffle(self.population)

            fitnesses = []
            for i in range(populationSize):
                fitness = self.calcFitness(self.population[i])
                if fitness == 1:
                    return self.population[i]
                fitnesses.append(fitness)

            sorted_population = list(zip(self.population, fitnesses))
            sorted_population.sort(key=lambda x: x[1], reverse=True)

            # find the best chromosomes based on their fitnesses, and carry them directly to the next generation (optional)
            carry_pivot = int(populationSize * carryPercentage)
            carriedChromosomes, _ = zip(*deepcopy(sorted_population[:carry_pivot]))

            # A pool consisting of potential candidates for mating (crossover and mutation)
            matingPool = self.createMatingPool(sorted_population)

            # The pool consisting of chromosomes after crossover
            crossoverPool = self.createCrossoverPool(matingPool)

            # Delete the previous population
            self.population.clear()

            # Create the portion of population that is undergone crossover and mutation
            mutatedPool = self.createMutatedPool(crossoverPool)
            self.population = mutatedPool[:len(self.population) - carry_pivot]

            # Add the prominent chromosomes directly to next generation
            self.population.extend(carriedChromosomes)


    def createMatingPool(self, pool):
        matingPool = []
        mate_list, mate_fitnesses = zip(*pool)
        total_fitness = sum(mate_fitnesses)
        probability = [x/total_fitness for x in mate_fitnesses]
        cumulative_prob = list(accumulate(probability))

        for _ in pool:
            rand = random.random()
            i = bisect.bisect_left(cumulative_prob, rand)
            matingPool.append(mate_list[i])
        return matingPool


    def createCrossoverPool(self, matingPool):
        crossoverPool = deepcopy(matingPool)
        length = len(crossoverPool)
        r1 = random.randint(0, length - 1)
        r2 = random.randint(r1, length)
        
        for i in range(len(crossoverPool)):
            if random.random() < crossoverProbability:
                to_change = crossoverPool[i][r1:r2]
                crossoverPool[i][r1:r2] = crossoverPool[(i + 1) % length][r1:r2]
                crossoverPool[(i + 1) % length][r1:r2] = to_change

        return crossoverPool


    def createMutatedPool(self, crossoverPool):
        mutated_pool = deepcopy(crossoverPool)
        for c in mutated_pool:
            if random.random() < mutationProbability:
                self.mutate(c)
        return mutated_pool


    def mutate(self, chromome):
        i = random.randrange(len(chromome))
        chromome[i] = str(random.choice(self.operators)) if i % 2 else str(random.choice(self.operands))


    def calcFitness(self, chromosome):
        diff = abs(eval(''.join(chromosome)) - self.goalNumber)
        return 1 / (diff + 1)


In [3]:
operands = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
operators = ['+', '-', '*', '%']
equationLength = 23
goalNumber = -123456

equationBuilder = EquationBuilder(operators, operands, equationLength, goalNumber)

tic = time.time()
equation = equationBuilder.findEquation()
tac = time.time()

if equation:
    equation = ''.join(equation)
    print(f'{equation} = {eval(equation)}')
else:
    print("Equation wasn't solved")

print(f'Executation time: {tac - tic}s')

<div style="font-size:200%" dir='rtl'>
سوالات
</div>
<br>

<div dir="RTL">
<ol>
  <li>
اگر جمعیت اولیه بسیار کم باشد، ممکن است آن ویژگی های خوب مورد نیاز را در کروموزوم ها نداشته باشیم. برعکسش اگر جمعیت اولیه بسیار زیاد باشد، محاسبات الگوریتم زیاد میشود و به تبع آن حافظه و زمان بیشتری مصرف میکند.
  </li>
  <br>

  <li
  >تنوع و تعداد کروموزوم ها بیشتر میشود. ولی از طرف دیگر محاسبات هم سنگین تر میشود و همینطور ممکن است همگرایی داده ها از بین برود.
  </li>
  <br>

  <li>کراس اور 2  کروموزوم را میگیرد و بعضی از ژن هایشان را جابجا میکند تا با ترکیب 2 جواب ترکیب بهتری را بسازد. ولی mutate به صورت مستقیم اطلاعات یک کروموزوم را تغییر میدهد تا همواره کروموزوم های جدیدی داشته باشیم و در نسلی از کروموزوم ها متوقف نشویم.
  </li>
  <br>

  <li
  >انتخاب مقدار مناسب برای پارامترهای مسئله و تعریف خوب تابع fitness.
  </li>
  <br>

  <li>یکی از دلایل ممکن است کم بودن
  mutationProbability
  باشد. این اتفاق موجب میشود. دیگر فیتنس زیاد نشود و به جواب درست نرسیم.
  برای حل این مشکل میتوان 
  mutationProbability
  را افزایش داد و در کنارش چند بار الگوریتم را اجرا کرد.
  </li>
  <br>

  <li>
  میتوان محدودیت زمان یا محدودیت نسل گذاشت که من از روش دوم استفاده کردم.
  </li>
  <br>

</ol>




<div style="font-size:100%" dir='rtl'>

</div>
</div>

